In [1]:
import sys
!conda install --yes --prefix {sys.prefix} rectorch

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - rectorch

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [2]:
from rectorch.models import RecSysModel, TorchNNTrainer, AETrainer, VAE, MultiDAE, MultiVAE,\
    CMultiVAE, EASE, CFGAN, ADMM_Slim, SVAE


import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import scipy
from scipy import sparse 
#from surprise import Reader, Dataset
#from surprise.model_selection import train_test_split
#from surprise import NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, SVD, NMF, accuracy
from collections import defaultdict

In [3]:
# initialize data
item_threshold = 1 # used to filter out user/artist pairs that have been 
                   #listened to less than the threshold number of times
popular_artist_fraction = 0.2 # top cutoff for what we consider popular artists, in this case the top 20%

user_events_file = "../data/user_events.txt"
low_user_file = "../data/low_main_users.txt"
medium_user_file = "../data/medium_main_users.txt/"
high_user_file = "../data/high_main_users.txt"


#read in user events file
cols = ['user', 'artist', 'album', 'track', 'timestamp']
df_events = pd.read_csv(user_events_file, sep='\t', names=cols)
print('No. of user events: ' + str(len(df_events)))
df_events.head() # check it is all read in properly


# create unique user-artist matrix
df_events = df_events.groupby(['user', 'artist']).size().reset_index(name='listens')
print('No. user-artist pairs: ' + str(len(df_events)))
# each row contains a unique user-artist pair, along with how many times the
# user has listened to the artist
df_events.head()

No. of user events: 28718087
No. user-artist pairs: 1755361


,user,artist,listens
0,1021445,12,43
1,1021445,16,1
2,1021445,28,7
3,1021445,29,1
4,1021445,46,1


In [4]:
#df_events.dtypes

artist_to_user = scipy.sparse.csr_matrix(df_events, dtype='float64')


model = EASE()
model.train(artist_to_user)